In [ ]:
# percentiles computation
import pandas as pd

# Load the CSV file
df = pd.read_csv('combined_data_9gauges.csv')

# Filter data where 'VALUE' is greater than 0
filtered_df = df[df['VALUE'] > 0].copy()  # Create a copy to avoid the SettingWithCopyWarning

# Convert the 'DATETIME' column to datetime type
df.loc[filtered_df.index, 'DATETIME'] = pd.to_datetime(filtered_df['DATETIME'])

# Calculate percentiles
percentiles = [50, 90, 97.5, 99.8, 99.9, 100]
percentile_values = filtered_df['VALUE'].quantile(q=[p/100 for p in percentiles])

# Print percentiles
for percentile, value in zip(percentiles, percentile_values):
    print(f"{percentile}th percentile: {value}")

In [ ]:
#percentiles plot across seasons and timescales

import matplotlib.pyplot as plt

# Dati di esempio
stagioni = ['Winter', 'Spring', 'Summer', 'Autumn']
percentili = ['50%', '90%', '97.5%', '99.8%', '99.9%', '100%']
scale_temporali = ['10 min', '1 Hr', '6 Hr', '12 Hr', '24 Hr']

# Supponiamo di avere i seguenti dati per ciascuna scala temporale
dati = {
    'Winter': {
        '10 min': [0.4, 1.0, 2.0, 6.2, 6.4, 16.6],
        '1 Hr': [0.6, 3.0, 6.0, 9.8, 12.0, 33.0],
        '6 Hr': [1.0, 7.8, 16.4, 34.2, 39.1, 56.2],
        '12 Hr': [1.4, 10.8, 23.0, 52.8, 60.4, 98.0],
        '24 Hr': [2.0, 15.2, 31.2, 75.8, 92.9, 165.2],
    },
    'Spring': {
        '10 min': [0.2, 1.0, 2.4, 9.6, 13.1, 26.9],
        '1 Hr': [0.6, 2.8, 5.6, 9.8, 11.0, 23.0],
        '6 Hr': [1.0, 7.2, 14.6, 28.0, 32.4, 43.6],
        '12 Hr': [1.4, 9.8, 18.8, 41.1, 48.2, 62.8],
        '24 Hr': [1.8, 12.8, 24.8, 55.5, 71.4, 101.2],
    },
    'Summer': {
        '10 min': [0.4, 2.2, 6.6, 16.4, 19.0, 25.6],
        '1 Hr': [0.6, 4.2, 8.2, 24.9, 33.0, 85.0],
        '6 Hr': [0.8, 7.8, 14.8, 39.2, 44.6, 94.0],
        '12 Hr': [1.0, 9.2, 17.6, 43.1, 46.1, 94.0],
        '24 Hr': [1.2, 10.7, 19.2, 51.7, 57.9, 94.0],
    },
    'Autumn': {
        '10 min': [0.4, 1.6, 4.2, 13.0, 15.2, 26.6],
        '1 Hr': [0.6, 4.2, 7.8, 19.7, 25.0, 71.0],
        '6 Hr': [1.2, 9.6, 18.4, 36.6, 44.1, 102.2],
        '12 Hr': [1.6, 12.6, 24.2, 51.1, 56.4, 135.2],
        '24 Hr': [2.2, 16.8, 30.6, 63.8, 66.6, 144.4],
    },
}

# Colori per le stagioni
colori = {
    'Winter': 'b',
    'Spring': 'g',
    'Summer': '#FFA500',
    'Autumn': '#FF0000',
}

# Creare il grafico con dimensioni maggiori
fig, ax = plt.subplots(figsize=(15, 10))

for stagione in stagioni:
    for scala in scale_temporali:
        x_values = [f"{scala} - {percentile}" for percentile in percentili]
        y_values = dati[stagione][scala]
        ax.plot(x_values, y_values, marker='o', color=colori[stagione], label=stagione)

# Aggiungere titolo e etichette con dimensioni maggiori
plt.title('Percentiles across seasons and time scales', fontsize=20)
#plt.xlabel('Percentiles and time scales', fontsize=16)
plt.ylabel('Value [mm]', fontsize=16)

# Aumentare dimensione etichette degli assi
plt.xticks(rotation=90, fontsize=12)
plt.yticks(fontsize=12)

# Aggiungere la leggenda fuori dal grafico con dimensioni maggiori
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)
plt.grid(True)
# Aggiungere layout regolato
plt.tight_layout()

plt.savefig('PAL1/Percentiles across seasons and time scales.jpg', format='jpg', dpi=300)


# Mostrare il grafico
plt.show()

In [ ]:
#PDF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# Reading your dataset and defining your helper functions
df = pd.read_csv('combined_data_9gauges.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

def classify_rainfall(value):
    filters = [(0.2, 0.4, 'WEAK'), (0.4001, 1.2, 'MODERATE'), 
               (1.2001, 2.8, 'HEAVY'), (2.8001, 10.4, 'VERY HEAVY'), 
               (10.401, 13, 'SHOWER'), (13.001, 2000, 'TORRENTIAL')]
    for lower, upper, label in filters:
        if lower <= value <= upper:
            return label
    return None

# Filtering for station 706
df = df[df['STATION'] == 706]
df['SEASON'] = df['DATETIME'].dt.month.apply(get_season)

# Define intervals and categories
intervals = [(2002, 2012), (2013, 2023)]
categories = ['WEAK', 'MODERATE', 'HEAVY', 'VERY HEAVY', 'SHOWER', 'TORRENTIAL']

# Processing data
event_data = {interval: {category: [] for category in categories} for interval in intervals}

for start_year, end_year in intervals:
    mask = df['DATETIME'].dt.year.between(start_year, end_year)
    temp_df = df[mask]

    event_duration = 0
    max_value = 0

    for index, row in temp_df.iterrows():
        if row['VALUE'] > 0:
            event_duration += 1
            max_value = max(max_value, row['VALUE'])
        else:
            if event_duration > 0:
                event_class = classify_rainfall(max_value)
                if event_class:
                    event_data[(start_year, end_year)][event_class].append(event_duration)
                event_duration = 0
                max_value = 0

    # Add the last event if it doesn't end with zero
    if event_duration > 0:
        event_class = classify_rainfall(max_value)
        if event_class:
            event_data[(start_year, end_year)][event_class].append(event_duration)

# Plotting PDFs
fig, axes = plt.subplots(len(intervals), len(categories), figsize=(21, 15), sharex=True, sharey=True)
for i, (start_year, end_year) in enumerate(intervals):
    for j, category in enumerate(categories):
        ax = axes[i, j]
        durations = event_data[(start_year, end_year)][category]
        
        if len(durations) > 0:
            density = gaussian_kde(durations)
            xs = np.linspace(0, max(durations), 200)
            density.covariance_factor = lambda : .25
            density._compute_covariance()
            ax.plot(xs, density(xs), label=category)
            ax.fill_between(xs, density(xs), alpha=0.5)

        ax.set_title(f"{category} ({start_year}-{end_year})")
        ax.set_xlabel('Duration')
        ax.set_ylabel('Density')
        ax.set_ylim(0,0.2)
        ax.set_xlim(0,100)

plt.tight_layout()
plt.show()

In [ ]:
# seasonal categorical donut percentages mm 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib.patches as mpatches

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

# Define a function to map months to seasons
def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

# Extract the year, month, and season from the 'DATETIME' column
df['Year'] = df['DATETIME'].dt.year
df['Month'] = df['DATETIME'].dt.month
df['Season'] = df['Month'].apply(map_month_to_season)

# Filters
filters = [
    (1, 2, 'WEAK'),
    (2.001, 6, 'MODERATE'),
    (6.001, 10, 'HEAVY'),
    (10.001, 2000, 'EXTREME')
]

# Define colors and year ranges
colors = ['paleturquoise', 'lightblue', 'indianred', 'salmon', 'mediumblue', 'lime']
year_ranges = [(2002, 2012), (2013, 2023)]


# Calculate the percentage of precipitation in each category
for min_range, max_range, category in filters:
    df[category] = ((df['VALUE'] >= min_range) & (df['VALUE'] <= max_range)).astype(int)

# Group by station, year, season, and sum the precipitation categories
grouped = df.groupby(['STATION', 'Year', 'Season']).agg({
    'WEAK': 'sum',
    'MODERATE': 'sum',
    'HEAVY': 'sum',
    'EXTREME': 'sum'
}).reset_index()

# Calculate the total percentage of precipitation for each station, year, and season combination
grouped['Total_Percentage'] = grouped[['WEAK', 'MODERATE', 'HEAVY', 'EXTREME']].sum(axis=1)

# Normalize the percentages
for category in filters:
    category_name = category[2]
    grouped[f'Normalized_{category_name}'] = grouped[category_name] / grouped['Total_Percentage']

fig, axs = plt.subplots(1, 2, figsize=(16, 16))
fig.subplots_adjust(hspace=0.5)

# Define legend patches
legend_patches = [mpatches.Patch(color=color, label=label) for color, label in zip(colors, [category[2] for category in filters])]

for ax, (start_year, end_year) in zip(axs, year_ranges):
    regional_df = grouped[grouped['Year'].between(start_year, end_year)]
    
    data = []
    labels = [category[2] for category in filters]
    
    for season in ['Spring', 'Summer', 'Autumn', 'Winter']:
        seasonal_df = regional_df[regional_df['Season'] == season]
        summed_df = seasonal_df.groupby(['Season'])[['Normalized_WEAK', 'Normalized_MODERATE', 'Normalized_HEAVY','Normalized_EXTREME']].sum().reset_index()
        
        if not summed_df.empty:
            data.append(summed_df.iloc[0, 1:].tolist())
        else:
            data.append([0] * len(filters))
    
    for i, season_data in enumerate(data):
        size = 0.2
        wedges, texts, autotexts = ax.pie(season_data, radius=1 - i * size, colors=colors, autopct='%1.1f%%', pctdistance=0.85, wedgeprops=dict(width=size, edgecolor='w'))
        plt.setp(autotexts, size=9)
    
    ax.set_title(f'({start_year}-{end_year})')
    
    # After plotting all seasons
    ax.legend(handles=legend_patches, loc='upper left')
    
    # Add a slightly increased value to pctdistance to prevent overlapping
    wedges, texts, autotexts = ax.pie(season_data, radius=1 - i * size, colors=colors, autopct='%1.1f%%', pctdistance=0.85, wedgeprops=dict(width=size, edgecolor='w'))
    plt.setp(autotexts, size=9)

fig.savefig("75gaugedonut_plot.jpg", dpi=300)
plt.show()


In [ ]:
# seasonal not extreme precipitation mm bar plot

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

# Define a function to map months to seasons
def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

# Extract the year, month, and season from the 'DATETIME' column
df['Year'] = df['DATETIME'].dt.year
df['Month'] = df['DATETIME'].dt.month
df['Season'] = df['Month'].apply(map_month_to_season)

# Filters
filters = [
    (1, 2, 'WEAK'),
    (2.001, 6, 'MODERATE'),
    (6.001,10, 'HEAVY'),
    (10.001, 2000, 'EXTREME')
]

# Calculate the total millimeters of precipitation in each category
for min_range, max_range, category in filters:
    df[category] = ((df['VALUE'] >= min_range) & (df['VALUE'] <= max_range)).astype(float) * df['VALUE']


# Group by station, year, season, and sum the precipitation categories
grouped = df.groupby(['STATION', 'Year', 'Season']).agg({
    'WEAK': 'sum',
    'MODERATE': 'sum',
    'HEAVY': 'sum',
    'EXTREME': 'sum'
}).reset_index()

# Define colors and year ranges
year_ranges = [(2002, 2012), (2013, 2023)]

fig, axs = plt.subplots(2, 1, figsize=(11, 7))
fig.subplots_adjust(hspace=1.5)

# Define colors and width
bar_width = 0.15
colors = ['paleturquoise','lightblue', 'indianred', 'salmon']

# Loop through year ranges
for ax, (start_year, end_year) in zip(axs, year_ranges):
    regional_df = grouped[grouped['Year'].between(start_year, end_year)]

    labels = ['Spring', 'Summer', 'Autumn', 'Winter']
    data = {label: [] for label in labels}

    for season in labels:
        seasonal_df = regional_df[regional_df['Season'] == season]
        for category in filters:
            category_name = category[2]
            category_total_mm = seasonal_df[category_name].sum()
            data[season].append(category_total_mm)

    x = np.arange(len(labels))
    for i, (category, color) in enumerate(zip(filters, colors)):
        category_name = category[2]
        y = [data[season][i] for season in labels]
        ax.bar(x + i * bar_width, y, color=color, width=bar_width, label=category_name)

        # Calculate the vertical position of annotations relative to the bars
        y_annotation = [value + 20 for value in y]  # You can adjust the value here for proper placement

        # Add y-values as text annotations on top of the bars
        for j, value in enumerate(y):
            ax.text(x[j] + i * bar_width, y_annotation[j], f'{value:.0f}', ha='center', fontsize=9, color='black')

    ax.set_xticks(x + bar_width * (len(filters) - 1) / 2)
    ax.set_xticklabels(labels)
    ax.set_ylim(0, 130000)
    ax.legend(loc='center', bbox_to_anchor=(0.5, 1.37), ncol=4)

fig.savefig("75gauge not extreme bar volumes.jpg", dpi=300)
plt.show()

In [ ]:
#seasonal categorical extreme precipitation mm bar plot

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

# Define a function to map months to seasons
def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

# Extract the year, month, and season from the 'DATETIME' column
df['Year'] = df['DATETIME'].dt.year
df['Month'] = df['DATETIME'].dt.month
df['Season'] = df['Month'].apply(map_month_to_season)

# Filters
filters = [
    (10.001, 18, 'VERY HEAVY'),
    (18.001, 25, 'SHOWER'),
    (25.001, 2000, 'TORRENTIAL')
]

# Calculate the total millimeters of precipitation in each category
for min_range, max_range, category in filters:
    df[category] = ((df['VALUE'] >= min_range) & (df['VALUE'] <= max_range)).astype(float) * df['VALUE']


# Group by station, year, season, and sum the precipitation categories
grouped = df.groupby(['STATION', 'Year', 'Season']).agg({
    'VERY HEAVY': 'sum',
    'SHOWER': 'sum',
    'TORRENTIAL': 'sum'
}).reset_index()


# Define colors and year ranges
year_ranges = [(2002, 2012), (2013, 2023)]

fig, axs = plt.subplots(2, 1, figsize=(11, 7))
fig.subplots_adjust(hspace=1.5)

# Define colors and width
bar_width = 0.15
colors = ['crimson', 'mediumblue', 'lime']

# Loop through year ranges
for ax, (start_year, end_year) in zip(axs, year_ranges):
    regional_df = grouped[grouped['Year'].between(start_year, end_year)]

    labels = ['Spring', 'Summer', 'Autumn', 'Winter']
    data = {label: [] for label in labels}

    for season in labels:
        seasonal_df = regional_df[regional_df['Season'] == season]
        for category in filters:
            category_name = category[2]
            category_total_mm = seasonal_df[category_name].sum()
            data[season].append(category_total_mm)

    x = np.arange(len(labels))
    for i, (category, color) in enumerate(zip(filters, colors)):
        category_name = category[2]
        y = [data[season][i] for season in labels]
        ax.bar(x + i * bar_width, y, color=color, width=bar_width, label=category_name)

        # Calculate the vertical position of annotations relative to the bars
        y_annotation = [value + 20 for value in y]  # You can adjust the value here for proper placement

        # Add y-values as text annotations on top of the bars
        for j, value in enumerate(y):
            ax.text(x[j] + i * bar_width, y_annotation[j], f'{value:.0f}', ha='center', fontsize=9, color='black')

    ax.set_xticks(x + bar_width * (len(filters) - 1) / 2)
    ax.set_xticklabels(labels)
    ax.set_ylim(0, 7000)
    ax.legend(loc='center', bbox_to_anchor=(0.5, 1.37), ncol=4)

fig.savefig("75gauge extreme bar volumes.jpg", dpi=300)
plt.show()

In [ ]:
#seasonal rainfall events vs duration histogram

import pandas as pd
import matplotlib.pyplot as plt

# Define custom colors for categories
category_colors = {
    'WEAK': 'lightcyan',
    'MODERATE': 'lightblue',
    'HEAVY': 'peachpuff',
    'VERY HEAVY': 'crimson',
    'SHOWER': 'mediumblue',
    'TORRENTIAL': 'lime'
}

def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

def classify_rainfall(value):
    filters = [(1, 2, 'WEAK'), (2.001, 6, 'MODERATE'), 
               (6.001, 10, 'HEAVY'), (10.001, 18, 'VERY HEAVY'), 
               (18.001, 25, 'SHOWER'), (25.001, 2000, 'TORRENTIAL')]
    
    for lower, upper, label in filters:
        if lower <= value <= upper:
            return label
    return None

df = pd.read_csv('75gauges.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['SEASON'] = df['DATETIME'].dt.month.apply(get_season)


# Filtering station 706
#df = df[df['STATION'] == 706]

# Splitting into two intervals
intervals = [(2002, 2012), (2013, 2023)]
categories = ['WEAK', 'MODERATE', 'HEAVY', 'VERY HEAVY', 'SHOWER', 'TORRENTIAL']

# Create a grid of subplots
fig, axes = plt.subplots(len(['Spring', 'Summer', 'Autumn', 'Winter']), len(intervals), figsize=(21, 15), sharex=True)

for i, season in enumerate(['Spring', 'Summer', 'Autumn', 'Winter']):
    for j, (start_year, end_year) in enumerate(intervals):
        ax = axes[i, j]
        mask = (df['DATETIME'].dt.year.between(start_year, end_year)) & (df['SEASON'] == season)
        temp_df = df[mask].copy()

        histogram = {}
        event_duration = 0
        max_value = 0
        
        for index, row in temp_df.iterrows():
            if row['VALUE'] > 0:
                event_duration += 1
                max_value = max(max_value, row['VALUE'])
            else:
                if event_duration > 0:
                    event_class = classify_rainfall(max_value)
                    histogram.setdefault(event_duration, {}).setdefault(event_class, 0)
                    histogram[event_duration][event_class] += 1
                    event_duration = 0
                    max_value = 0
                    
        if event_duration > 0:
            event_class = classify_rainfall(max_value)
            histogram.setdefault(event_duration, {}).setdefault(event_class, 0)
            histogram[event_duration][event_class] += 1
        
        durations = sorted(histogram.keys())
        data = {category: [] for category in categories}

        for duration in durations:
            for category in categories:
                data[category].append(histogram[duration].get(category, 0))

        df_plot = pd.DataFrame(data, index=durations)

        # Create a list of colors corresponding to your categories
        colors = [category_colors[category] for category in categories]

        # Plot on the respective subplot with custom colors
        df_plot.plot(kind='bar', stacked=True, ax=ax, color=colors)
        ax.set_title(f"{season} ({start_year}-{end_year})")
        ax.set_xlabel("Duration (hours)")
        ax.set_ylabel("Number of Events")
        #ax.set_ylim(0,100)
        #ax.legend(title='Rainfall Intensity', loc='upper right')
        plt.xticks(rotation=0)
        
        # Set y-axis limits for each season
        if season == 'Spring':
            ax.set_ylim(0, 4000)  # Adjust the limits as needed for each season
        elif season == 'Summer':
            ax.set_ylim(0, 2000)
        elif season == 'Autumn':
            ax.set_ylim(0, 7000)
        elif season == 'Winter':
            ax.set_ylim(0, 7000)
        #ax.legend()
        
        # Set black edge color for each bar
        for container in ax.containers:
            for bar in container:
                bar.set_edgecolor('k')

# Adjust layout and show the plots
plt.tight_layout()
fig.savefig("75gauge all durations.jpg", dpi=300)
plt.show()

In [ ]:
#seasonal rainfall events vs duration histogram - focus on heavy events

import pandas as pd
import matplotlib.pyplot as plt

# Define custom colors for categories
category_colors = {
    'HEAVY': 'peachpuff'
}

def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

def classify_rainfall(value):
    filters = [ (6.001, 10, 'HEAVY')]
    
    for lower, upper, label in filters:
        if lower <= value <= upper:
            return label
    return None

df = pd.read_csv('75gauges.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['SEASON'] = df['DATETIME'].dt.month.apply(get_season)



# Splitting into two intervals
intervals = [(2002, 2012), (2013, 2023)]
categories = ['HEAVY']

# Create a grid of subplots
fig, axes = plt.subplots(len(['Spring', 'Summer', 'Autumn', 'Winter']), len(intervals), figsize=(21, 15), sharex=True)

# Define y limits for each season
y_limits = {
    'Spring': 25,   # Adjust these values as needed
    'Summer': 60,
    'Autumn': 120,
    'Winter': 18
}

for i, season in enumerate(['Spring', 'Summer', 'Autumn', 'Winter']):
    for j, (start_year, end_year) in enumerate(intervals):
        ax = axes[i, j]
        mask = (df['DATETIME'].dt.year.between(start_year, end_year)) & (df['SEASON'] == season)
        temp_df = df[mask].copy()

        histogram = {}
        event_duration = 0
        max_value = 0
        
        for index, row in temp_df.iterrows():
            if row['VALUE'] > 0:
                event_duration += 1
                max_value = max(max_value, row['VALUE'])
            else:
                if event_duration > 0:
                    event_class = classify_rainfall(max_value)
                    histogram.setdefault(event_duration, {}).setdefault(event_class, 0)
                    histogram[event_duration][event_class] += 1
                    event_duration = 0
                    max_value = 0
                    
        if event_duration > 0:
            event_class = classify_rainfall(max_value)
            histogram.setdefault(event_duration, {}).setdefault(event_class, 0)
            histogram[event_duration][event_class] += 1
        
        durations = sorted(histogram.keys())
        data = {category: [] for category in categories}

        for duration in durations:
            for category in categories:
                data[category].append(histogram[duration].get(category, 0))

        df_plot = pd.DataFrame(data, index=durations)

        # Create a list of colors corresponding to your categories
        colors = [category_colors[category] for category in categories]

        # Plot on the respective subplot with custom colors
        df_plot.plot(kind='bar', stacked=True, ax=ax, color=colors)
        ax.set_title(f"{season} ({start_year}-{end_year})")
        ax.set_xlabel("Duration (hours)")
        ax.set_ylabel("Number of Events")
        #ax.set_ylim(0, y_limits[season])  # Set y limit based on the season
        #ax.legend(title='Rainfall Intensity', loc='upper right')
        plt.xticks(rotation=0)
        
        # Set black edge color for each bar
        for container in ax.containers:
            for bar in container:
                bar.set_edgecolor('k')

# Adjust layout and show the plots
plt.tight_layout()
#fig.savefig("75gauge_extreme_durations.jpg", dpi=300)

plt.show()


In [ ]:
#seasonal rainfall events vs duration histogram - focus on extreme events
import pandas as pd
import matplotlib.pyplot as plt

# Define custom colors for categories
category_colors = {
    'VERY HEAVY': 'crimson',
    'SHOWER': 'mediumblue',
    'TORRENTIAL': 'lime'
}

def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

def classify_rainfall(value):
    filters = [ (10.001, 18, 'VERY HEAVY'), 
               (18.001, 25, 'SHOWER'), (25.001, 2000, 'TORRENTIAL')]
    
    for lower, upper, label in filters:
        if lower <= value <= upper:
            return label
    return None

df = pd.read_csv('75gauges.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['SEASON'] = df['DATETIME'].dt.month.apply(get_season)



# Splitting into two intervals
intervals = [(2002, 2012), (2013, 2023)]
categories = ['VERY HEAVY', 'SHOWER', 'TORRENTIAL']

# Create a grid of subplots
fig, axes = plt.subplots(len(['Spring', 'Summer', 'Autumn', 'Winter']), len(intervals), figsize=(21, 15), sharex=True)

# Define y limits for each season
y_limits = {
    'Spring': 25,   # Adjust these values as needed
    'Summer': 60,
    'Autumn': 120,
    'Winter': 18
}

for i, season in enumerate(['Spring', 'Summer', 'Autumn', 'Winter']):
    for j, (start_year, end_year) in enumerate(intervals):
        ax = axes[i, j]
        mask = (df['DATETIME'].dt.year.between(start_year, end_year)) & (df['SEASON'] == season)
        temp_df = df[mask].copy()

        histogram = {}
        event_duration = 0
        max_value = 0
        
        for index, row in temp_df.iterrows():
            if row['VALUE'] > 0:
                event_duration += 1
                max_value = max(max_value, row['VALUE'])
            else:
                if event_duration > 0:
                    event_class = classify_rainfall(max_value)
                    histogram.setdefault(event_duration, {}).setdefault(event_class, 0)
                    histogram[event_duration][event_class] += 1
                    event_duration = 0
                    max_value = 0
                    
        if event_duration > 0:
            event_class = classify_rainfall(max_value)
            histogram.setdefault(event_duration, {}).setdefault(event_class, 0)
            histogram[event_duration][event_class] += 1
        
        durations = sorted(histogram.keys())
        data = {category: [] for category in categories}

        for duration in durations:
            for category in categories:
                data[category].append(histogram[duration].get(category, 0))

        df_plot = pd.DataFrame(data, index=durations)

        # Create a list of colors corresponding to your categories
        colors = [category_colors[category] for category in categories]

        # Plot on the respective subplot with custom colors
        df_plot.plot(kind='bar', stacked=True, ax=ax, color=colors)
        ax.set_title(f"{season} ({start_year}-{end_year})")
        ax.set_xlabel("Duration (hours)")
        ax.set_ylabel("Number of Events")
        ax.set_ylim(0, y_limits[season])  # Set y limit based on the season
        #ax.legend(title='Rainfall Intensity', loc='upper right')
        plt.xticks(rotation=0)
        
        # Set black edge color for each bar
        for container in ax.containers:
            for bar in container:
                bar.set_edgecolor('k')

# Adjust layout and show the plots
plt.tight_layout()
fig.savefig("75gauge_extreme_durations.jpg", dpi=300)

plt.show()


In [ ]:
#max/h versus rainfall depth plus duration
import matplotlib.pyplot as plt
import pandas as pd

def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

def classify_rainfall(value):
    filters = [(1, 2, 'WEAK'), (2.001, 6, 'MODERATE'),
               (6.001, 10, 'HEAVY'), (10.001, 18, 'VERY HEAVY'),
               (18.001, 25, 'SHOWER'), (25.001, 2000, 'TORRENTIAL')]
    for lower, upper, label in filters:
        if lower <= value <= upper:
            return label
    return None

# Replace this with the actual path to your 'gauges.csv' file
df = pd.read_csv('75gauges.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['SEASON'] = df['DATETIME'].dt.month.apply(get_season)

# Filtering station 706
#df = df[df['STATION'] == 706]

# Splitting into two intervals
intervals = [(2002, 2012), (2013, 2023)]

# Define custom colors for categories
category_colors = {
    'WEAK': 'paleturquoise',
    'MODERATE': 'lightblue',
    'HEAVY': 'peachpuff',
    'VERY HEAVY': 'crimson',
    'SHOWER': 'mediumblue',
    'TORRENTIAL': 'lime'
}

# Define custom markers for duration ranges
duration_markers = {
    '<=6': 'o',        # Circle marker for <=6 hours
    '>6 <=12': '^',    # Square marker for >6 and <=12 hours
    '>12 <=24': 's',   # Triangle marker for >12 and <=24 hours
    '>24 <=48': 'D',   # Diamond marker for >24 and <=48 hours
    '>48': 'P'         # Pentagon marker for >48 hours
}

# Create a grid of subplots with seasons as rows and intervals as columns
fig, axes = plt.subplots(len(['Spring', 'Summer', 'Autumn', 'Winter']), len(intervals), figsize=(15, 10), sharex='col')

for i, season in enumerate(['Spring', 'Summer', 'Autumn', 'Winter']):
    for j, (start_year, end_year) in enumerate(intervals):
        mask = (df['DATETIME'].dt.year.between(start_year, end_year)) & (df['SEASON'] == season)
        temp_df = df[mask].copy()

        total_rainfall = []
        max_values = []
        duration_ranges = []  # Initialize list for duration ranges
        event_duration = 0
        total_value = 0
        max_value = 0

        for index, row in temp_df.iterrows():
            if row['VALUE'] > 0:
                event_duration += 1
                total_value += row['VALUE']
                max_value = max(max_value, row['VALUE'])
            else:
                if event_duration > 0:
                    total_rainfall.append(total_value)
                    max_values.append(max_value)

                    # Classify the duration range
                    if event_duration <= 6:
                        duration_ranges.append('<=6')
                    elif event_duration <= 12:
                        duration_ranges.append('>6 <=12')
                    elif event_duration <= 24:
                        duration_ranges.append('>12 <=24')
                    elif event_duration <= 48:
                        duration_ranges.append('>24 <=48')
                    else:
                        duration_ranges.append('>48')

                    # Reset for the next event
                    event_duration = 0
                    total_value = 0
                    max_value = 0

        if event_duration > 0:
            total_rainfall.append(total_value)
            max_values.append(max_value)

            # Classify the duration range for the last event
            if event_duration <= 6:
                duration_ranges.append('<=6')
            elif event_duration <= 12:
                duration_ranges.append('>6 <=12')
            elif event_duration <= 24:
                duration_ranges.append('>12 <=24')
            elif event_duration <= 48:
                duration_ranges.append('>24 <=48')
            else:
                duration_ranges.append('>48')

        # Classify the maximum rainfall values into categories
        max_value_categories = [classify_rainfall(value) for value in max_values]

        # Map categories to colors and markers, and plot with markers
        ax = axes[i, j]
        for category, color in category_colors.items():
            for marker_range, marker in duration_markers.items():
                combined_mask = [
                    cat == category and duration == marker_range
                    for cat, duration in zip(max_value_categories, duration_ranges)
                ]
                ax.scatter(
                    [total_rainfall[k] for k, v in enumerate(combined_mask) if v],
                    [max_values[k] for k, v in enumerate(combined_mask) if v],
                    c=color,
                    edgecolor='k',
                    label=f"{season} ({start_year}-{end_year}) - {category}",
                    marker=marker,
                    alpha=0.7,  # Adjust alpha for better visibility of overlapping points
                )

        ax.set_title(f"{season} ({start_year}-{end_year})")
        ax.set_xlabel("Rainfall depth (mm)")
        ax.set_ylabel("Maximum Rainfall Value")
        ax.set_yscale('log')
        ax.set_xscale('log')
        ax.set_ylim(1, 131)
        ax.set_xlim(1, 400)

# Adjust layout and show the plots
plt.tight_layout()
fig.savefig("75gaugemaxvsrainfalldepthLOGLOG.jpg", dpi=300)# Adjust the space to avoid overlap

plt.show()

In [ ]:
#max/h versus rainfall depth plus number of events
import matplotlib.pyplot as plt
import pandas as pd

def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

def classify_rainfall(value):
    filters = [(1, 2, 'WEAK'), (2.001, 6, 'MODERATE'),
               (6.001, 10, 'HEAVY'), (10.001, 18, 'VERY HEAVY'),
               (18.001, 25, 'SHOWER'), (25.001, 2000, 'TORRENTIAL')]

    for lower, upper, label in filters:
        if lower <= value <= upper:
            return label
    return None

df = pd.read_csv('75gauges.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['SEASON'] = df['DATETIME'].dt.month.apply(get_season)

# Filtering station 706
#df = df[df['STATION'] == 706]

# Splitting into two intervals
intervals = [(2002, 2012), (2013, 2023)]

# Define custom colors for categories
category_colors = {
    'WEAK': 'paleturquoise',
    'MODERATE': 'lightblue',
    'HEAVY': 'peachpuff',
    'VERY HEAVY': 'crimson',
    'SHOWER': 'mediumblue',
    'TORRENTIAL': 'lime'
}

# Create a grid of subplots with seasons as rows and intervals as columns
fig, axes = plt.subplots(len(['Spring', 'Summer', 'Autumn', 'Winter']), len(intervals), figsize=(15, 10), sharex='col')

for i, season in enumerate(['Spring', 'Summer', 'Autumn', 'Winter']):
    for j, (start_year, end_year) in enumerate(intervals):
        mask = (df['DATETIME'].dt.year.between(start_year, end_year)) & (df['SEASON'] == season)
        temp_df = df[mask].copy()

        total_rainfall = []
        max_values = []
        event_duration = 0
        total_value = 0
        max_value = 0

        for index, row in temp_df.iterrows():
            if row['VALUE'] > 0:
                event_duration += 1
                total_value += row['VALUE']
                max_value = max(max_value, row['VALUE'])
            else:
                if event_duration > 0:
                    total_rainfall.append(total_value)
                    max_values.append(max_value)
                    event_duration = 0
                    total_value = 0
                    max_value = 0

        if event_duration > 0:
            total_rainfall.append(total_value)
            max_values.append(max_value)

        # Classify the maximum rainfall values into categories
        max_value_categories = [classify_rainfall(value) for value in max_values]

        # Map categories to colors and plot with 'o' markers
        ax = axes[i, j]
        scatter_counts = {}  # Dictionary to store scatter point counts for each category
        for category, color in category_colors.items():
            cat_mask = [cat == category for cat in max_value_categories]
            x_values = [total_rainfall[k] for k, v in enumerate(cat_mask) if v]
            y_values = [max_values[k] for k, v in enumerate(cat_mask) if v]

            # Count the number of scatter points
            scatter_count = len(x_values)
            scatter_counts[category] = scatter_count

            ax.scatter(x_values, y_values, c=color, label=f"{season} ({start_year}-{end_year}) - {category}")

        ax.set_title(f"{season} ({start_year}-{end_year})")
        ax.set_xlabel("Rainfall depth (mm)")
        ax.set_ylabel("Maximum Rainfall Value")
        ax.set_yscale('log')
        #ax.set_xscale('log')

        # Annotate the total scatter point count on the plot
        total_count_text = '\n'.join([f"{category}: {count}" for category, count in scatter_counts.items()])
        ax.annotate(total_count_text, xy=(0.1, 0.8), xycoords='axes fraction', fontsize=10, color='black')



# Adjust layout and show the plots
plt.tight_layout()
fig.savefig("75gaugemaxvsrainfalldepthANNOT.jpg", dpi=300)  # Adjust the space to avoid overlap
plt.show()

In [ ]:
#rainfall depth vs duration

import matplotlib.pyplot as plt
import pandas as pd

def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

def classify_rainfall(value):
    filters = [(1, 2, 'WEAK'), (2.001, 6, 'MODERATE'),
               (6.001, 10, 'HEAVY'), (10.001, 18, 'VERY HEAVY'),
               (18.001, 25, 'SHOWER'), (25.001, 2000, 'TORRENTIAL')]

    for lower, upper, label in filters:
        if lower <= value <= upper:
            return label
    return None

df = pd.read_csv('75gauges.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['SEASON'] = df['DATETIME'].dt.month.apply(get_season)


# Filtering station 706
#df = df[df['STATION'] == 706]

# Splitting into two intervals
intervals = [(2002, 2012), (2013, 2023)]

# Create subplots for each season as rows and intervals as columns
fig, axes = plt.subplots(len(['Spring', 'Summer', 'Autumn', 'Winter']), len(intervals), figsize=(15, 10), sharex='col')

for i, season in enumerate(['Spring', 'Summer', 'Autumn', 'Winter']):
    for j, (start_year, end_year) in enumerate(intervals):
        durations = []
        total_rainfall = []
        max_values = []  # Store maximum values for each duration
        event_duration = 0
        total_value = 0
        max_value = 0

        mask = (df['DATETIME'].dt.year.between(start_year, end_year)) & (df['SEASON'] == season)
        temp_df = df[mask].copy()

        for index, row in temp_df.iterrows():
            if row['VALUE'] > 0:
                event_duration += 1
                total_value += row['VALUE']
                max_value = max(max_value, row['VALUE'])
            else:
                if event_duration > 0:
                    total_rainfall.append(total_value)
                    durations.append(event_duration)
                    max_values.append(max_value)  # Append the maximum value for this duration
                    event_duration = 0
                    total_value = 0
                    max_value = 0

        if event_duration > 0:
            total_rainfall.append(total_value)
            durations.append(event_duration)
            max_values.append(max_value)  # Append the maximum value for the last duration

        # Classify the maximum values into categories
        max_value_categories = [classify_rainfall(value) for value in max_values]

        # Define custom colors for categories
        category_colors = {
            'WEAK': 'paleturquoise',
            'MODERATE': 'lightblue',
            'HEAVY': 'peachpuff',
            'VERY HEAVY': 'crimson',
            'SHOWER': 'mediumblue',
            'TORRENTIAL': 'lime'
        }

        # Map categories to colors and plot with 'o' markers
        ax = axes[i, j]
        for category, color in category_colors.items():
            cat_mask = [cat == category for cat in max_value_categories]
            ax.scatter([durations[k] for k, v in enumerate(cat_mask) if v],
                       [total_rainfall[k] for k, v in enumerate(cat_mask) if v],
                       c=color,
                       label=f"{season} ({start_year}-{end_year}) - {category}")

        ax.set_title(f"{season} ({start_year}-{end_year})")
        ax.set_xlabel("Duration (hours)")
        ax.set_ylabel("Rainfall depth (mm)")
        ax.set_yscale('log')
        ax.set_xscale('log')
        ax.set_ylim(1,400)
        #ax.legend()

# Adjust layout and show the plots
plt.tight_layout()
fig.savefig("75gaugedurationvsrainfalldepthLOGLOG.jpg", dpi=300)# Adjust the space to avoid overlap

plt.show()


In [ ]:
#simple ratio

import numpy as np
import matplotlib.pyplot as plt

# Define colors and year ranges
year_ranges = [(2002, 2012)]

# Define colors and width per category
colors = {
    'Volumes': 'blue',
    'Events': 'grey'
}
bar_width = 0.20

# Define the filters for the categories
filters = [
    (1, 2, 'Volumes'),
    (2.001, 6, 'Events')
]

# Define the y-values for the bars for each time interval (replace with your desired values)
data_per_interval = [
    {
        'WEAK': [0.89, 0.91],  # Replace with your values for the first interval
        'MODERATE': [0.90, 0.94],  # Replace with your values for the first interval
        'HEAVY': [0.95, 1.01],  # Replace with your values for the first interval
        'VERY HEAVY': [0.97, 0.98],  # Replace with your values for the first interval
        'SHOWER': [1.13, 1.20],  # Replace with your values for the first interval
        'TORRENTIAL': [1.41, 1.28]
    }
]

# Define the labels for the seasons
season_labels = ['WEAK', 'MODERATE', 'HEAVY', 'VERY HEAVY', 'SHOWER', 'TORRENTIAL']

x = np.arange(len(season_labels))

for idx, (start_year, end_year) in enumerate(year_ranges):
    fig, ax = plt.subplots(figsize=(7, 2))
    current_data = data_per_interval[idx]

    for i, (category, color) in enumerate(zip(filters, colors.values())):
        category_name = category[2]
        y_values = [current_data[season][i] for season in season_labels]
        bars = ax.bar(x + i * bar_width, y_values, color=color, width=bar_width, label=f'{category_name}')
        
        # Annotate the y-values on top of the bars with fontsize
        fontsize = 7  # Adjust the fontsize as needed
        for bar, y in zip(bars, y_values):
            ax.text(bar.get_x() + bar.get_width() / 2, y + 0.05, f'{y:.2f}', ha='center', va='bottom', fontsize=fontsize)

    ax.set_xticks(x + (len(filters) - 1) * bar_width / 2)
    ax.set_xticklabels(season_labels)  # Label x-axis with season names
    ax.set_ylim(0, 2)  # Adjust the y-axis limit as needed
    ax.set_title(f'Simple ratio of volumes and events per intensity class')
    
    # Set the size (linewidth) of the dashed horizontal line
    dashed_line_size = 0.5  # Adjust the line size as needed
    ax.axhline(y=1, color='red', linestyle='--', label='Y=1', linewidth=dashed_line_size)
    
    plt.tight_layout()
    fig.savefig("75gaugeSimple ratio of volumes and events per intensity class.jpg", dpi=300)
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define colors and year ranges
year_ranges = [(2002, 2012), (2013, 2023)]

# Define colors and width per category
colors = {
    'WEAK': 'paleturquoise',
    'MODERATE': 'lightblue',
    'HEAVY': 'indianred',
    'VERY HEAVY': 'crimson',
    'SHOWER': 'mediumblue',
    'TORRENTIAL': 'lime'
}
bar_width = 0.15

# Define the filters for the categories
filters = [
    (1, 2, 'WEAK'),
    (2.001, 6, 'MODERATE'),
    (6.001, 10, 'HEAVY'),
    (10.001, 18, 'VERY HEAVY'),
    (18.001, 30, 'SHOWER'),
    (30.001, 2000, 'TORRENTIAL')
]

# Define the y-values for the bars for each time interval (replace with your desired values)
data_per_interval = [
    {
        'Spring': [0.93, 0.91, 0.95, 0.99, 0.61, 1.02],  # Replace with your values for the first interval
        'Summer': [1.14, 1.33, 1.67, 1.55, 2.88, 2.32],  # Replace with your values for the first interval
        'Autumn': [0.99, 1.03, 1.08, 1.05, 1.38, 1.31],  # Replace with your values for the first interval
        'Winter': [0.81, 0.76, 0.67, 0.59, 0.35, 1.13],  # Replace with your values for the first interval
    },
    {
        'Spring': [1.07, 0.99, 0.97, 0.97, 0.72, 1],  # Replace with your values for the first interval
        'Summer': [1.21, 1.37, 1.65, 1.58, 3.09, 2.14],  # Replace with your values for the first interval
        'Autumn': [0.89, 0.94, 1.09, 1.04, 1.40, 1.21],  # Replace with your values for the first interval
        'Winter': [0.81, 0.85, 0.75, 0.61, 0.35, 0.91],  # Replace with your values for the first interval
    }
]

# Define the labels for the seasons
season_labels = ['Spring', 'Summer', 'Autumn', 'Winter']

# Define custom titles for each plot
custom_titles = [
    'Simple ratio of volumes per season per intensity class ',
    'Simple ratio of events per season per intensity class'
]

x = np.arange(len(season_labels))

# Create a single figure with two subplots
fig, axes = plt.subplots(2, 1, figsize=(7, 4))

for idx, (start_year, end_year) in enumerate(year_ranges):
    ax = axes[idx]
    current_data = data_per_interval[idx]

    for i, (category, color) in enumerate(zip(filters, colors.values())):
        category_name = category[2]
        y_values = [current_data[season][i] for season in season_labels]
        bars = ax.bar(x + i * bar_width, y_values, color=color, width=bar_width, label=f'{category_name}')

        # Annotate the y-values on top of the bars with a specified fontsize
        fontsize = 6  # Adjust the fontsize as needed
        for bar, y in zip(bars, y_values):
            ax.text(bar.get_x() + bar.get_width() / 2, y + 0.1, f'{y:.2f}', ha='center', va='bottom', fontsize=fontsize)

    ax.set_xticks(x + (len(filters) - 1) * bar_width / 2)
    ax.set_xticklabels(season_labels)  # Label x-axis with season names
    ax.set_ylim(0, 4)  # Adjust the y-axis limit as needed
    ax.set_title(custom_titles[idx])
    ax.axhline(y=1, color='red', linestyle='--', label='Y=1', linewidth=0.5)  # Add horizontal line at Y=1

# Adjust layout
plt.tight_layout()

# Save the figure
fig.savefig(f"75gaugeSimple_ratio_per_season_intensity_class.jpg", dpi=300)

# Show the figure
plt.show()

In [ ]:
#sicily map

import folium
import pandas as pd

# Read data from the COO CSV file
df = pd.read_csv("merged_COO.csv")

# Create a map centered around a location (you can choose any location)
m = folium.Map(location=[37.5, 14], zoom_start=8)

# Define a dictionary to map locations to marker colors
location_colors = {
    'CATANIA': 'red',
    'PALERMO': 'red',
    'TRAPANI FONTANASALSA': 'red',
    'AGRIGENTO SCIBICA': 'red',
    'CALTANISSETTA': 'red',
    'MESSINA': 'red',
    'ENNA': 'red',
    'RAGUSA': 'red',
    'SIRACUSA': 'red'
}

# Iterate through the DataFrame and add markers with different colors for specific locations
for index, row in df.iterrows():
    if not pd.isnull(row["EST"]) and not pd.isnull(row["NORD"]):
        location = row["LOCATION"]
        color = location_colors.get(location, 'black')  # Default to blue if location is not in the dictionary
        folium.CircleMarker(
            location=[float(row["NORD"]), float(row["EST"])],
            radius=3,  # Adjust the radius as needed
            popup=location,
            fill=True,
            fill_color=color,  # Use the color from the dictionary
            color=color,
            fill_opacity=0.6,
        ).add_to(m)

# Save the map to an HTML file
m


In [ ]:
#monthly counts trend plot

import csv
import matplotlib.pyplot as plt
import calendar
import numpy as np

def plot_data(csv_file1, csv_file2, csv_file3, csv_file4, degree=1):
    month_data = {}  # Store data per month

    # Initialize month_data dictionary
    for month in range(1, 13):
        month_data[month] = {
            'year': [],
            'weak_counts': [],
            'moderate_counts': [],
            'heavy_counts': [],
            'severe_counts': [],
            'negative_sums': [],
            'monthly_sums': []
        }

    with open(csv_file1, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row

        for row in reader:
            month_year = row[0]
            month = int(month_year.split('-')[1])
            weak_count = int(row[1])
            moderate_count = int(row[2])
            heavy_count = int(row[3])
            severe_count = int(row[4])

            month_data[month]['year'].append(month_year.split('-')[0])
            month_data[month]['weak_counts'].append(weak_count)
            month_data[month]['moderate_counts'].append(moderate_count)
            month_data[month]['heavy_counts'].append(heavy_count)
            month_data[month]['severe_counts'].append(severe_count)

    with open(csv_file2, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row

        for row in reader:
            month_year = row[0]
            month = int(month_year.split('-')[1])
            negative_sum = float(row[2])

            month_data[month]['negative_sums'].append(negative_sum)

    with open(csv_file3, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row

        for row in reader:
            month_year = row[0]
            month = int(month_year.split('-')[1])
            monthly_sum = float(row[1])

            month_data[month]['monthly_sums'].append(monthly_sum)

    fig, axs = plt.subplots(12, 1, figsize=(15, 47))  # Create subplots

    for month in range(1, 13):
        ax = axs[month - 1]  # Select the appropriate subplot
        data = month_data[month]
        year = data['year']
        weak_counts = data['weak_counts']
        moderate_counts = data['moderate_counts']
        heavy_counts = data['heavy_counts']
        severe_counts = data['severe_counts']
        negative_sums = data['negative_sums']
        monthly_sums = data['monthly_sums']

        # Plot the trendlines
        categories = ['weak_counts', 'moderate_counts', 'heavy_counts', 'severe_counts']
        colors = ['blue', 'green', 'red', 'purple']
        for category, color in zip(categories, colors):
            counts = data[category]
            if len(counts) > 1:
                counts_np = np.array(counts)
                trend = np.polyfit(np.arange(len(counts_np)), counts_np, degree)
                ax.plot(year, np.polyval(trend, np.arange(len(counts_np))), color=color, linestyle='--', label=f'Trend - {category}')
                slope = round(trend[-2], 2)
                ax.text(year[-1], np.polyval(trend, len(counts_np)-1), f'Slope: {slope}', color=color)

        ax.set_xlabel('Year')
        ax.set_ylabel('Count')
        ax.set_title(f'Monthly Counts for {calendar.month_name[month]}')
        ax.grid(True)
        #ax.set_ylim(-1, 2)

        ax.set_xticks(year)
        ax.set_xticklabels(year, rotation='vertical')

        ax.legend()

    plt.tight_layout()
    #plt.savefig('monthly_plots_trendlines_slopes.jpg', quality=350)

# CSV file paths
csv_file1 = '2370_reprocessed.csv'
csv_file2 = 'rgmonthly_sums.csv'
csv_file3 = 'rgmonthlysum.csv'
csv_file4 = 'rgyearlymm.csv'

# Plot the data per month and save as individual plots
plot_data(csv_file1, csv_file2, csv_file3, csv_file4, degree=1)